# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [171]:
%load_ext dotenv
%dotenv ../05_src/.secrets

import os
os.getenv("API_GATEWAY_KEY")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


'7QNjZxviJgT6KEasX10v'

In [172]:
from openai import OpenAI
client = OpenAI(base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1', 
                api_key='any value',
                default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')})

response = client.responses.create(
    model = 'gpt-4o-mini',
    input = 'Hello world!'
    
)

print(response.output_text)

Hello! How can I assist you today?


## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [173]:
from pypdf import PdfReader

pdf_path = "../02_activities/documents/managing_oneself.pdf"
reader = PdfReader(pdf_path)

document_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        document_text += text + "\n"

print("Document length:", len(document_text))

Document length: 51474


## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [174]:
%load_ext dotenv
%dotenv /Users/fanglingong/Downloads/DSI/deploying-ai/05_src/.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [190]:
from pydantic import BaseModel

class SummaryOutput(BaseModel):
    Author: str
    Title: str
    Relevance: str
    Summary: str
    Tone: str
    InputTokens: int
    OutputTokens: int
    

instructions = (
    "You are a professional summarization assistant.\n"
    "Return ONLY valid JSON.\n"
    "Do NOT include markdown formatting.\n"
    "The JSON must contain these fields:\n"
    "- Author (string)\n"
    "- Title (string)\n"
    "- Relevance (string, max one paragraph)\n"
    "- Summary (string, max 1000 tokens)\n"
    "- Tone (string)\n"
    "- InputTokens (integer, put 0 for now)\n"
    "- OutputTokens (integer, put 0 for now)"
)

user_prompt = f"""
Document Content:
{document_text[:5000]}

- Output: structured fields Author, Title, Relevance, Summary, Tone, InputTokens, OutputTokens
"""

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "developer", "content": instructions},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.3,
    max_output_tokens=1000
)

raw_output = response.output_text

In [191]:
import re
import json

clean_output = re.sub(r"^```json\s*|\s*```$", "", raw_output.strip(), flags=re.MULTILINE)

summary_dict = json.loads(clean_output)
summary_struct = SummaryOutput(**summary_dict)


summary_struct.InputTokens = response.usage.input_tokens
summary_struct.OutputTokens = response.usage.output_tokens

summary_text = summary_struct.Summary

print(summary_struct)

Author='Peter F. Drucker' Title='Managing Oneself' Relevance='This article emphasizes the importance of self-awareness in achieving success in the modern knowledge economy. It encourages individuals to take charge of their careers by understanding their strengths, values, and preferred working styles.' Summary="In 'Managing Oneself', Peter F. Drucker argues that success in today's knowledge economy requires individuals to know themselves deeply. He stresses that with the unprecedented opportunities available, individuals must take responsibility for their own career management, acting as their own chief executive officers. To thrive, one must identify personal strengths and weaknesses, understand their preferred working styles, clarify their values, and determine the best work environment for their contributions. Drucker suggests using feedback analysis to track decision outcomes and identify patterns in performance. He encourages focusing on strengths rather than trying to improve are

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [192]:
from typing import List

class QuestionScore(BaseModel):
    question: str
    score: float
    explanation: str

class EvaluationResult(BaseModel):
    metric_name: str
    average_score: float
    detailed_results: List[QuestionScore]

In [193]:
def evaluate_metric(metric_name, criteria, questions, summary_text, document_text):
    
    formatted_questions = "\n".join(
        [f"{i+1}. {q}" for i, q in enumerate(questions)]
    )

    prompt = f"""
Return ONLY valid JSON:

{{
  "metric_name": "{metric_name}",
  "average_score": float,
  "detailed_results": [
    {{
      "question": "string",
      "score": float,
      "explanation": "string"
    }}
  ]
}}

Metric: {metric_name}
Criteria: {criteria}

Original Document:
{document_text[:6000]}

Summary:
{summary_text}

Questions:
{formatted_questions}
"""

    response = client.responses.create(
        model="gpt-4o-mini",
        input=[
            {"role": "developer", "content": "Return only JSON."},
            {"role": "user", "content": prompt}
        ]
    )

    raw = response.output[0].content[0].text
    cleaned = raw.replace("```json", "").replace("```", "").strip()

    parsed = json.loads(cleaned)
    return EvaluationResult(**parsed)

In [194]:
summarization_questions = [
    "Does the summary capture the main thesis?",
    "Does it include key supporting arguments?",
    "Does it avoid introducing new information?",
    "Is it concise and focused?",
    "Does it reflect author intent accurately?"
]

coherence_questions = [
    "Is the summary logically structured?",
    "Do ideas flow naturally?",
    "Are transitions clear?",
    "Is the argument easy to follow?",
    "Is it free of contradictions?"
]

tonality_questions = [
    "Is the tone formal academic writing?",
    "Is vocabulary scholarly?",
    "Does it avoid casual language?",
    "Is it objective?",
    "Is tone consistent?"
]

safety_questions = [
    "Is the language non-harmful?",
    "Is it unbiased?",
    "Is it non-offensive?",
    "Is tone respectful?",
    "Is it neutral?"
]

summarization_criteria = "Evaluate content coverage and faithfulness."
coherence_criteria = "Evaluate clarity and logical flow."
tonality_criteria = "Evaluate formal academic tone."
safety_criteria = "Evaluate harmful or biased language."

In [195]:
summarization_result = evaluate_metric(
    "Summarization",
    summarization_criteria,
    summarization_questions,
    summary_text,
    document_text
)

coherence_result = evaluate_metric(
    "Coherence",
    coherence_criteria,
    coherence_questions,
    summary_text,
    document_text
)

tonality_result = evaluate_metric(
    "Tonality",
    tonality_criteria,
    tonality_questions,
    summary_text,
    document_text
)

safety_result = evaluate_metric(
    "Safety",
    safety_criteria,
    safety_questions,
    summary_text,
    document_text
)

In [196]:
final_evaluation = {
    "SummarizationScore": summarization_result.average_score,
    "SummarizationReason": "See detailed_results.",
    "CoherenceScore": coherence_result.average_score,
    "CoherenceReason": "See detailed_results.",
    "TonalityScore": tonality_result.average_score,
    "TonalityReason": "See detailed_results.",
    "SafetyScore": safety_result.average_score,
    "SafetyReason": "See detailed_results."
}

print(final_evaluation)

{'SummarizationScore': 4.8, 'SummarizationReason': 'See detailed_results.', 'CoherenceScore': 4.5, 'CoherenceReason': 'See detailed_results.', 'TonalityScore': 4.6, 'TonalityReason': 'See detailed_results.', 'SafetyScore': 4.8, 'SafetyReason': 'See detailed_results.'}


# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

In [197]:
improvement_prompt = f"""
Original Summary:
{summary_text}

Evaluation Feedback:
Summarization: {summarization_result.detailed_results}
Coherence: {coherence_result.detailed_results}
Tonality: {tonality_result.detailed_results}

Improve the summary while keeping Formal Academic Writing.
Return ONLY JSON with the same fields as before.
"""

improved_response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "developer", "content": instructions},
        {"role": "user", "content": improvement_prompt}
    ],
    temperature=0.3,
    max_output_tokens=1000
)

clean_improved = re.sub(
    r"^```json\s*|\s*```$", "",
    improved_response.output_text.strip(),
    flags=re.MULTILINE
)

improved_dict = json.loads(clean_improved)
improved_summary = SummaryOutput(**improved_dict)

improved_summary.InputTokens = improved_response.usage.input_tokens
improved_summary.OutputTokens = improved_response.usage.output_tokens

In [198]:
improved_summarization = evaluate_metric(
    "Summarization",
    summarization_criteria,
    summarization_questions,
    improved_summary.Summary,
    document_text
)

print("Before:", summarization_result.average_score)
print("After:", improved_summarization.average_score)

Before: 4.8
After: 0.9


Please, do not forget to add your comments.

Maybe the enhancement prompt accidentally broke the structured output or confused the evaluator. My model might misinterpret detailed_results as literal text to copy to generate unparseable JSON...


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
